<a href="https://colab.research.google.com/github/Cirediallo/Models/blob/main/icefall_asr_tedlium3_pretrained_transducer_stateless_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Description
This notebook shows how to use a pre-trained Transducer Stateless model with [icefall](https://github.com/k2-fsa/icefall/).

## Environment setup

To use a pre-trained model with icefall, we have to install the following dependencies:

- [k2][k2], for FSA operations
- [torchaudio][audio], for reading sound files
- [kaldifeat][kaldifeat], for extracting features from a single sound
  file or multiple sound files

**NOTE**: [lhotse][lhotse] is used only in training time, for data preparation.


[k2]: https://github.com/k2-fsa/k2
[audio]: https://github.com/pytorch/audio
[kaldifeat]: https://github.com/csukuangfj/kaldifeat
[lhotse]: https://github.com/lhotse-speech/lhotse

In [7]:
import os

### Install PyTorch and torchaudio



In [34]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [35]:
! pip uninstall -y torch torchaudio torchvision torchtext

! pip install torch==1.7.1+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Found existing installation: torch 1.13.1
Uninstalling torch-1.13.1:
  Successfully uninstalled torch-1.13.1
Found existing installation: torchaudio 0.13.1
Uninstalling torchaudio-0.13.1:
  Successfully uninstalled torchaudio-0.13.1
Found existing installation: torchvision 0.15.1+cu118
Uninstalling torchvision-0.15.1+cu118:
  Successfully uninstalled torchvision-0.15.1+cu118
Found existing installation: torchtext 0.15.1
Uninstalling torchtext-0.15.1:
  Successfully uninstalled torchtext-0.15.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.7.1+cu101 (from versions: 1.11.0, 1.11.0+cpu, 1.11.0+cu102, 1.11.0+cu113, 1.11.0+cu115, 1.11.0+rocm4.3.1, 1.11.0+rocm4.5.2, 1.12.0, 1.12.0+cpu, 1.12.0+cu102, 1.12.0+cu113, 1.12.0+cu116, 1.12.0+rocm5.0, 1.12.0+rocm5.1.1, 1.12.1, 1.12.1+cpu, 1.12.1+cu102, 1.12.1+

### Install CUDA 11.7 => start execution here

In [1]:
! wget https://developer.download.nvidia.com/compute/cuda/11.7.1/local_installers/cuda_11.7.1_515.65.01_linux.run

--2023-05-01 10:49:44--  https://developer.download.nvidia.com/compute/cuda/11.7.1/local_installers/cuda_11.7.1_515.65.01_linux.run
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3524358811 (3.3G) [application/octet-stream]
Saving to: ‘cuda_11.7.1_515.65.01_linux.run’

cuda_11.7.1_515.65. 100%[===================>]   3.28G   271MB/s    in 13s     

2023-05-01 10:49:57 (266 MB/s) - ‘cuda_11.7.1_515.65.01_linux.run’ saved [3524358811/3524358811]



In [2]:
! chmod +x /content/cuda_11.7.1_515.65.01_linux.run

In [10]:
! cd /content/ && \
  ./cuda_11.7.1_515.65.01_linux.run \
    --silent \
    --toolkit \
    --installpath=/content/cuda11/software/cuda-11.7.1 \
    --no-opengl-libs \
    --no-drm \
    --no-man-page

#### Install cuDNN for CUDA 11.7

In [11]:
! wget https://huggingface.co/csukuangfj/cudnn/resolve/main/cudnn-11.3-linux-x64-v8.2.0.53.tgz

--2023-05-01 11:00:28--  https://huggingface.co/csukuangfj/cudnn/resolve/main/cudnn-11.3-linux-x64-v8.2.0.53.tgz
Resolving huggingface.co (huggingface.co)... 18.160.249.70, 18.160.249.78, 18.160.249.31, ...
Connecting to huggingface.co (huggingface.co)|18.160.249.70|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/csukuangfj/cudnn/7a195dc93a7cda2bdd4d9b73958d259c784be422cd941a9a625aab75309f19dc?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27cudnn-11.3-linux-x64-v8.2.0.53.tgz%3B+filename%3D%22cudnn-11.3-linux-x64-v8.2.0.53.tgz%22%3B&Expires=1683198029&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL2NzdWt1YW5nZmovY3Vkbm4vN2ExOTVkYzkzYTdjZGEyYmRkNGQ5YjczOTU4ZDI1OWM3ODRiZTQyMmNkOTQxYTlhNjI1YWFiNzUzMDlmMTlkYz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE2ODMxOTgwMjl9fX1dfQ__&Signature=ttQoIxYkP5V1aMT9jT-8CWzB2ghsvcwGb

In [12]:
! cd /content/ && tar xvf ./cudnn-11.3-linux-x64-v8.2.0.53.tgz --strip-components=1 -C /content/cuda11/software/cuda-11.7.1

cuda/include/cudnn.h
cuda/include/cudnn_adv_infer.h
cuda/include/cudnn_adv_infer_v8.h
cuda/include/cudnn_adv_train.h
cuda/include/cudnn_adv_train_v8.h
cuda/include/cudnn_backend.h
cuda/include/cudnn_backend_v8.h
cuda/include/cudnn_cnn_infer.h
cuda/include/cudnn_cnn_infer_v8.h
cuda/include/cudnn_cnn_train.h
cuda/include/cudnn_cnn_train_v8.h
cuda/include/cudnn_ops_infer.h
cuda/include/cudnn_ops_infer_v8.h
cuda/include/cudnn_ops_train.h
cuda/include/cudnn_ops_train_v8.h
cuda/include/cudnn_v8.h
cuda/include/cudnn_version.h
cuda/include/cudnn_version_v8.h
cuda/NVIDIA_SLA_cuDNN_Support.txt
cuda/lib64/libcudnn.so
cuda/lib64/libcudnn.so.8
cuda/lib64/libcudnn.so.8.2.0
cuda/lib64/libcudnn_adv_infer.so
cuda/lib64/libcudnn_adv_infer.so.8
cuda/lib64/libcudnn_adv_infer.so.8.2.0
cuda/lib64/libcudnn_adv_train.so
cuda/lib64/libcudnn_adv_train.so.8
cuda/lib64/libcudnn_adv_train.so.8.2.0
cuda/lib64/libcudnn_cnn_infer.so
cuda/lib64/libcudnn_cnn_infer.so.8
cuda/lib64/libcudnn_cnn_infer.so.8.2.0
cuda/lib64/

In [13]:
"""
! echo $PATH
print("****")
!pwd
print("========")
! echo $PYTHONPATH

print("--------")
#os.environ['PYTHONPATH'] += ":/content/icefall"

! echo $PYTHONPATH

"""
os.environ['CUDA_HOME'] = "/content/cuda11/software/cuda-11.7.1"
os.environ['PATH'] += ":/content/cuda11/software/cuda-11.7.1/bin"
os.environ['LD_LIBRARY_PATH'] += ":/content/cuda11/software/cuda-11.7.1/lib64"
os.environ['LD_LIBRARY_PATH'] += ":/content/cuda11/software/cuda-11.7.1/lib"

os.environ['CUDA_TOOLKIT_ROOT_DIR']=os.environ['CUDA_HOME']
os.environ['CUDA_TOOLKIT_ROOT']=os.environ['CUDA_HOME']
os.environ['CUDA_BIN_PATH']=os.environ['CUDA_HOME']
os.environ['CUDA_PATH']=os.environ['CUDA_HOME']
os.environ['CUDA_INC_PATH']="/content/cuda11/software/cuda-11.7.1/targets/x86_64-linux"
#os.environ['CFLAGS'] +="-I"+":/content/cuda11/software/cuda-11.7.1/targets/x86_64-linux/include"


In [17]:
! python3 -m k2.version


k2 version: 1.24.1
Build type: Release
Git SHA1: 7b67d26bb0e1d4b51044e00eb043cc87a6bfbc85
Git date: Sun Apr 30 05:50:05 2023
Cuda used to build k2: 11.7
cuDNN used to build k2: 8.2.0
Python version used to build k2: 3.10
OS used to build k2: Ubuntu 20.04.5 LTS
CMake version: 3.25.2
GCC version: 9.4.0
CMAKE_CUDA_FLAGS:  -Wno-deprecated-gpu-targets   -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_35,code=sm_35  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_50,code=sm_50  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_60,code=sm_60  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_61,code=sm_61  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_70,code=sm_70  -lineinfo --expt-extended-lambda -use_fast_math 

### Install k2

In [14]:
! pip install torch==1.13.1
! pip install torchaudio==0.13.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.6 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1+cu118 requires torch==2.0.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.1 requires torch==2.0.0, but you have torch

In [8]:
#! pip install k2==1.17

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached k2-1.17-py38-none-any.whl (72.7 MB)
ERROR: Could not find a version that satisfies the requirement torch==1.7.1 (from k2) (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0)
ERROR: No matching distribution found for torch==1.7.1


In [27]:
#! pip install k2==1.10.dev20211102+cuda10.1.torch1.7.1 -f https://k2-fsa.org/nightly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://k2-fsa.org/nightly
ERROR: Could not find a version that satisfies the requirement k2==1.10.dev20211102+cuda10.1.torch1.7.1 (from versions: 0.0.1.dev0, 0.0.2.dev0, 0.0.3.dev20201019, 0.0.3.dev20201020, 0.0.3.dev20201025, 0.0.3.dev20201027, 0.0.4.dev20201027, 0.0.5.dev20201027, 0.0.5.dev20201028, 0.0.6.dev20201028, 0.0.7.dev20201028, 0.0.9.dev20201109, 0.0.10.dev20201110, 0.1, 0.1.1.dev20201116, 0.1.1.dev20201118, 0.1.1.dev20201130, 0.1.1.dev20201208, 0.1.1.dev20201210, 0.1.1.dev20201212, 0.1.1.dev20201216, 0.1.2.dev20210111, 0.1.2, 0.1.3.dev20210111, 0.3.0, 0.3.1.dev20210121, 0.3.1.dev20210127, 0.3.1.dev20210204, 0.3.2.dev20210204, 0.3.3.dev20210205, 0.3.3.dev20210206, 0.3.3.dev20210209, 0.3.3.dev20210218, 0.3.3.dev20210222, 0.3.3.dev20210224, 0.3.3.dev20210302, 0.3.3.dev20210305, 0.3.3.dev20210309, 0.3.3.dev20210321, 0.3.3.dev20210328, 0.3.3.dev20210331, 0.3.3.de

Check that k2 was installed successfully:

In [15]:
! git clone https://github.com/k2-fsa/k2.git

Cloning into 'k2'...
remote: Enumerating objects: 14195, done.
remote: Counting objects: 100% (467/467), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 14195 (delta 349), reused 450 (delta 341), pack-reused 13728
Receiving objects: 100% (14195/14195), 15.38 MiB | 28.07 MiB/s, done.
Resolving deltas: 100% (9920/9920), done.


In [16]:
!cd /content/k2 && \
  export K2_MAKE_ARGS="-j6" && \
  python3 setup.py install

running install
/usr/lib/python3.10/distutils/cmd.py:62: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/lib/python3.10/distutils/cmd.py:62: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://github.com/pypa/setuptools/issues/917

### Install kaldifeat

In [18]:
! KALDIFEAT_MAKE_ARGS="-j" pip install --verbose kaldifeat

Using pip 23.0.1 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 482.4/482.4 kB 10.3 MB/s eta 0:00:00
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-yq1ut5cb/kaldifeat.egg-info
  writing /tmp/pip-pip-egg-info-yq1ut5cb/kaldifeat.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-yq1ut5cb/kaldifeat.egg-info/dependency_links.txt
  writing top-level names to /tmp/pip-pip-egg-info-yq1ut5cb/kaldifeat.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-yq1ut5cb/kaldifeat.egg-info/SOURCES.txt'
  reading manifest file '/tmp/pip-pip-egg-info-yq1ut5cb/kaldifeat.egg-info/SOURCES.txt'
  reading manifest template 'MANIFEST.in'
  adding license file 'LICENSE'
  writing manifest file '/tmp/pip-pip-egg-info-yq1ut5cb/kaldifeat.egg-info/SOURCES.txt'
  Preparing meta

To check that kaldifeat was installed successfully, run

In [19]:
! python3 -c "import kaldifeat; print(kaldifeat.__version__)"

1.21


### Install icefall

icefall is a collection of Python scripts. All you need is just to
download its source code and set the `PYTHONPATH` environment variable.

In [20]:
! git clone https://github.com/k2-fsa/icefall

Cloning into 'icefall'...
remote: Enumerating objects: 11840, done.
remote: Counting objects: 100% (183/183), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 11840 (delta 89), reused 92 (delta 44), pack-reused 11657
Receiving objects: 100% (11840/11840), 13.61 MiB | 32.86 MiB/s, done.
Resolving deltas: 100% (8048/8048), done.


In [21]:
! pip install -q kaldialign sentencepiece>=0.1.96 lhotse

In [22]:
! pip install git+https://github.com/lhotse-speech/lhotse


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/lhotse-speech/lhotse to /tmp/pip-req-build-1nsgaai9
  Running command git clone --filter=blob:none --quiet https://github.com/lhotse-speech/lhotse /tmp/pip-req-build-1nsgaai9
  Resolved https://github.com/lhotse-speech/lhotse to commit cbd7b2b3e57b0e97470fe015eca5eafa6bb85d5e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for lhotse: filename=lhotse-1.15.0.dev0+git.cbd7b2b.clean-py3-none-any.whl size=639690 sha256=469583cf46557159ca3d61e49ecbdb287b15b74eca6c5df7a28f772959a9bfd8
  Stored in directory: /tmp/pip-ephem-wheel-cache-yg2t400g/wheels/df/b0/ff/cce0f16868fcdbee2088f3acf9f249dc90117d5f5dd9b6f69d
Successfully built lhotse
  Attempting uninstall: lhotse
    Found existing installation: lhotse 1.14.0
    Uninstalling lhotse-1.14.0:
      Successfully unin

In [23]:
# Python
from lhotse.tools import install_sph2pipe
install_sph2pipe()

/root/.lhotse/tools/sph2pipe-2.5.tar.gz: 0.00B [00:00, ?B/s]

## Download pre-trained RNN-T Conformer model

To make the following steps easier, we decide to download the model
to `icefall/egs/tedlium3/ASR/tmp`

In [ ]:
! apt-get install -y -qq tree sox git-lfs

**CAUTION**: You have to run `sudo apt-get install git-lfs`
and `git lfs install` in order to download the pre-trained model.

In [ ]:
! cd icefall/egs/tedlium3/ASR && rm -rf tmp

In [ ]:
! cd icefall/egs/tedlium3/ASR && \
  mkdir tmp && \
  cd tmp && \
  git lfs install && \
  git clone -v https://huggingface.co/luomingshuang/icefall_asr_tedlium3_transducer_stateless && \
  cd icefall_asr_tedlium3_transducer_stateless && \
  cd ../.. && \
  tree tmp

Updated git hooks.
Git LFS initialized.
Cloning into 'icefall_asr_tedlium3_transducer_stateless'...
POST git-upload-pack (165 bytes)
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 78 (delta 0), reused 4 (delta 0), pack-reused 74
Unpacking objects: 100% (78/78), 2.78 MiB | 2.50 MiB/s, done.
Filtering content: 100% (4/4), 2.21 GiB | 31.63 MiB/s, done.
tmp
└── icefall_asr_tedlium3_transducer_stateless
    ├── data
    │   └── lang_bpe_500
    │       └── bpe.model
    ├── exp
    │   ├── cpu_jit_epoch_19_avg_11_torch.1.7.1.pt
    │   ├── pretrained_average_14_to_29.pt
    │   └── pretrained_average_19_to_29.pt
    ├── log
    │   ├── beam_search
    │   │   ├── errs-dev-beam_4-epoch-29-avg-11-beam-4.txt
    │   │   ├── errs-dev-beam_4-epoch-29-avg-16-beam-4.txt
    │   │   ├── errs-test-beam_4-epoch-29-avg-11-beam-4.txt
    │   │   ├── errs-test-beam_4-epoch-29-avg-16-beam-4.txt
    │   │   ├─

In [ ]:
! soxi icefall/egs/tedlium3/ASR/tmp/icefall_asr_tedlium3_transducer_stateless/test_wavs/*.wav


Input File     : 'icefall/egs/tedlium3/ASR/tmp/icefall_asr_tedlium3_transducer_stateless/test_wavs/RichBenjamin_2015W01.wav'
Channels       : 1
Sample Rate    : 16000
Precision      : 16-bit
Duration       : 00:00:30.02 = 480256 samples ~ 2251.2 CDDA sectors
File Size      : 961k
Bit Rate       : 256k
Sample Encoding: 16-bit Signed Integer PCM


Input File     : 'icefall/egs/tedlium3/ASR/tmp/icefall_asr_tedlium3_transducer_stateless/test_wavs/RichBenjamin_2015W02.wav'
Channels       : 1
Sample Rate    : 16000
Precision      : 16-bit
Duration       : 00:00:30.02 = 480256 samples ~ 2251.2 CDDA sectors
File Size      : 961k
Bit Rate       : 256k
Sample Encoding: 16-bit Signed Integer PCM


Input File     : 'icefall/egs/tedlium3/ASR/tmp/icefall_asr_tedlium3_transducer_stateless/test_wavs/RichBenjamin_2015W03.wav'
Channels       : 1
Sample Rate    : 16000
Precision      : 16-bit
Duration       : 00:00:29.95 = 479232 samples ~ 2246.4 CDDA sectors
File Size      : 959k
Bit Rate       : 256k


## Greedy search

In [ ]:
! cd icefall/egs/tedlium3/ASR && \
    PYTHONPATH=/content/icefall python3 ./transducer_stateless/pretrained.py \
      --method greedy_search \
      --max-sym-per-frame 1 \
      --checkpoint ./tmp/icefall_asr_tedlium3_transducer_stateless/exp/pretrained_average_19_to_29.pt \
      --bpe-model ./tmp/icefall_asr_tedlium3_transducer_stateless/data/lang_bpe_500/bpe.model \
      ./tmp/icefall_asr_tedlium3_transducer_stateless/test_wavs/RichBenjamin_2015W01.wav \
      ./tmp/icefall_asr_tedlium3_transducer_stateless/test_wavs/RichBenjamin_2015W02.wav \
      ./tmp/icefall_asr_tedlium3_transducer_stateless/test_wavs/RichBenjamin_2015W03.wav

2023-04-17 14:59:34,785 INFO [pretrained.py:249] {'sample_rate': 16000, 'feature_dim': 80, 'encoder_out_dim': 512, 'subsampling_factor': 4, 'attention_dim': 512, 'nhead': 8, 'dim_feedforward': 2048, 'num_encoder_layers': 12, 'vgg_frontend': False, 'env_info': {'k2-version': '1.23.4', 'k2-build-type': 'Release', 'k2-with-cuda': True, 'k2-git-sha1': '62e404dd3f3a811d73e424199b3408e309c06e1a', 'k2-git-date': 'Mon Jan 30 02:26:16 2023', 'lhotse-version': '1.13.0', 'torch-version': '1.13.1+cu117', 'torch-cuda-available': True, 'torch-cuda-version': '11.7', 'python-version': '3.9', 'icefall-git-branch': 'master', 'icefall-git-sha1': '8838fe0-clean', 'icefall-git-date': 'Mon Apr 17 09:47:25 2023', 'icefall-path': '/content/icefall', 'k2-path': '/usr/local/lib/python3.9/dist-packages/k2/__init__.py', 'lhotse-path': '/usr/local/lib/python3.9/dist-packages/lhotse/__init__.py', 'hostname': '3f5f2b213a05', 'IP address': '172.28.0.12'}, 'checkpoint': './tmp/icefall_asr_tedlium3_transducer_stateless

## Beam search

In [ ]:
! cd icefall/egs/tedlium3/ASR && \
    PYTHONPATH=/content/icefall python3 ./transducer_stateless/pretrained.py \
      --method beam_search \
      --beam-size 4 \
      --checkpoint ./tmp/icefall_asr_tedlium3_transducer_stateless/exp/pretrained_average_19_to_29.pt \
      --bpe-model ./tmp/icefall_asr_tedlium3_transducer_stateless/data/lang_bpe_500/bpe.model \
      ./tmp/icefall_asr_tedlium3_transducer_stateless/test_wavs/RichBenjamin_2015W01.wav \
      ./tmp/icefall_asr_tedlium3_transducer_stateless/test_wavs/RichBenjamin_2015W02.wav \
      ./tmp/icefall_asr_tedlium3_transducer_stateless/test_wavs/RichBenjamin_2015W03.wav

2023-04-17 14:59:59,016 INFO [pretrained.py:249] {'sample_rate': 16000, 'feature_dim': 80, 'encoder_out_dim': 512, 'subsampling_factor': 4, 'attention_dim': 512, 'nhead': 8, 'dim_feedforward': 2048, 'num_encoder_layers': 12, 'vgg_frontend': False, 'env_info': {'k2-version': '1.23.4', 'k2-build-type': 'Release', 'k2-with-cuda': True, 'k2-git-sha1': '62e404dd3f3a811d73e424199b3408e309c06e1a', 'k2-git-date': 'Mon Jan 30 02:26:16 2023', 'lhotse-version': '1.13.0', 'torch-version': '1.13.1+cu117', 'torch-cuda-available': True, 'torch-cuda-version': '11.7', 'python-version': '3.9', 'icefall-git-branch': 'master', 'icefall-git-sha1': '8838fe0-clean', 'icefall-git-date': 'Mon Apr 17 09:47:25 2023', 'icefall-path': '/content/icefall', 'k2-path': '/usr/local/lib/python3.9/dist-packages/k2/__init__.py', 'lhotse-path': '/usr/local/lib/python3.9/dist-packages/lhotse/__init__.py', 'hostname': '3f5f2b213a05', 'IP address': '172.28.0.12'}, 'checkpoint': './tmp/icefall_asr_tedlium3_transducer_stateless

In [ ]:
! cd icefall/egs/tedlium3/ASR && \
    PYTHONPATH=/content/icefall python3 ./transducer_stateless/pretrained.py \
      --method beam_search \
      --beam-size 4 \
      --checkpoint ./tmp/icefall_asr_tedlium3_transducer_stateless/exp/pretrained_average_19_to_29.pt \
      --bpe-model ./tmp/icefall_asr_tedlium3_transducer_stateless/data/lang_bpe_500/bpe.model \
      /content/84-121123-0002.flac \
      /content/fr1.wav

2023-04-17 15:17:53,303 INFO [pretrained.py:249] {'sample_rate': 16000, 'feature_dim': 80, 'encoder_out_dim': 512, 'subsampling_factor': 4, 'attention_dim': 512, 'nhead': 8, 'dim_feedforward': 2048, 'num_encoder_layers': 12, 'vgg_frontend': False, 'env_info': {'k2-version': '1.23.4', 'k2-build-type': 'Release', 'k2-with-cuda': True, 'k2-git-sha1': '62e404dd3f3a811d73e424199b3408e309c06e1a', 'k2-git-date': 'Mon Jan 30 02:26:16 2023', 'lhotse-version': '1.13.0', 'torch-version': '1.13.1+cu117', 'torch-cuda-available': True, 'torch-cuda-version': '11.7', 'python-version': '3.9', 'icefall-git-branch': 'master', 'icefall-git-sha1': '8838fe0-clean', 'icefall-git-date': 'Mon Apr 17 09:47:25 2023', 'icefall-path': '/content/icefall', 'k2-path': '/usr/local/lib/python3.9/dist-packages/k2/__init__.py', 'lhotse-path': '/usr/local/lib/python3.9/dist-packages/lhotse/__init__.py', 'hostname': '3f5f2b213a05', 'IP address': '172.28.0.12'}, 'checkpoint': './tmp/icefall_asr_tedlium3_transducer_stateless

In [ ]:
! cd icefall/egs/tedlium3/ASR && \
  export CUDA_VISIBLE_DEVICES="0,1,2,3" && \
  python ./transducer_stateless/train.py  && \
  --world-size 4 && \
  --num-epochs 30 \
  --start-epoch 0 \
  --exp-dir transducer_stateless/exp \
  --max-duration 300

Traceback (most recent call last):
  File "/content/icefall/egs/tedlium3/ASR/./transducer_stateless/train.py", line 44, in <module>
    from asr_datamodule import TedLiumAsrDataModule
  File "/content/icefall/egs/tedlium3/ASR/transducer_stateless/asr_datamodule.py", line 37, in <module>
    from icefall.utils import str2bool
ModuleNotFoundError: No module named 'icefall'


## Train

In [24]:
! cd icefall && \
  ! pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.3 MB/s eta 0:00:00


In [25]:

#import os

!pwd
! echo $PYTHONPATH


os.environ['PYTHONPATH'] += ":/content/icefall"

! echo $PYTHONPATH

/content
/env/python
/env/python:/content/icefall


In [26]:
pip list  | grep torch

k2                            1.24.1.dev20230501+cuda11.7.torch1.13.1
torch                         1.13.1
torchaudio                    0.13.1
torchdata                     0.6.0
torchsummary                  1.5.1
torchtext                     0.15.1
torchvision                   0.15.1+cu118


In [ ]:
"""
torch                         1.13.1
torchaudio                    2.0.1+cu118
torchdata                     0.6.0
torchsummary                  1.5.1
torchtext                     0.15.1
torchvision                   0.15.1+cu118
"""

In [32]:
#pip install torchaudio==0.13.1

In [34]:
! cd icefall/egs/tedlium3/ASR && \
  ./prepare.sh

2023-05-01 14:55:16 (prepare.sh:50:main) dl_dir: /content/icefall/egs/tedlium3/ASR/download
2023-05-01 14:55:16 (prepare.sh:53:main) Stage 0: Download data
2023-05-01 14:55:16 (prepare.sh:83:main) Stage 1: Prepare tedlium3 manifests
2023-05-01 14:55:16 (prepare.sh:94:main) Stage 2: Prepare musan manifests
2023-05-01 14:55:16 (prepare.sh:105:main) Stage 3: Compute fbank for tedlium3
Traceback (most recent call last):
  File "/content/icefall/egs/tedlium3/ASR/./local/compute_fbank_tedlium.py", line 112, in <module>
    compute_fbank_tedlium()
  File "/content/icefall/egs/tedlium3/ASR/./local/compute_fbank_tedlium.py", line 73, in compute_fbank_tedlium
    extractor = KaldifeatFbank(KaldifeatFbankConfig(num_mel_bins=num_mel_bins))
NameError: name 'KaldifeatFbank' is not defined
